<a href="https://colab.research.google.com/github/lobird/machine-learning/blob/master/cnn_mnist_100acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


准备MNIST数据

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/',one_hot=True)
print(mnist.train.num_examples,mnist.validation.num_examples)
BATCH_SIZE = 128
print(mnist.train.images.shape,mnist.train.labels.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
55000 5000
(55000, 784) (55000, 10)


In [0]:
#参数设置
filter_size = 5
num_filter1 = 32
num_filter2 = 64
max_pool_size = 2
img_size = 28
img_shape = [28,28]
img_size_flat = 28*28
img_channel = 1
num_fc = 1024
num_class = 10
dropout = 0.75
LEARNING_RATE_BASE = 1e-4
LEARNING_RATE_DECAY = 1-(1e-5)
MODEL_SAVE_PATH = '/content/gdrive/My Drive/model_save/cnn_mnist/'
MODEL_NAME = 'cnn_mnist'
MOVING_AVERAGE_DECAY = 0.99

In [0]:
#definition of some function
def get_weights(shape):
  return tf.Variable(tf.random_normal(shape))
def get_biases(length):
  return tf.Variable(tf.random_normal([length]))

def conv_layer(d_input,
              num_input_channels,
              filter_size,
              num_filters,):
  shape = [filter_size,filter_size,num_input_channels,num_filters]
  weights = get_weights(shape)
  biases = get_biases(num_filters)
  layer_1 = tf.nn.conv2d(d_input,
                       filter=weights,
                       strides=[1,1,1,1],
                       padding='SAME' )
  layer_2 = layer_1 + biases
  layer_3 = tf.nn.max_pool(value=layer_2,
                        ksize=[1,max_pool_size,max_pool_size,1],
                        strides = [1,max_pool_size,max_pool_size,1],
                        padding='SAME')
  layer = tf.nn.relu(layer_3)
  return layer,weights

def flatten_layer(layer):
  layer_shape = layer.shape
  n = layer_shape[1:4].num_elements()
  layer_flat = tf.reshape(layer,[-1,n])
  return layer_flat,n

def fc_layer(input,
            num_inputs,
            num_outputs,):
  weights = get_weights(shape = [num_inputs,num_outputs])
  biases = get_biases(length = num_outputs )
  layer_fc = (tf.matmul(input,weights) + biases)
  return layer_fc


In [0]:
#definition of math structure:
def forward(input):
  
  x_img = tf.reshape(input,[-1,img_size,img_size,img_channel])
  layer1,w1 = conv_layer(x_img,
                        img_channel,
                        filter_size,
                        num_filter1)
  layer2,w2 = conv_layer(layer1,
                        num_filter1,
                        filter_size,
                        num_filter2)
  layer_flat,n_feature = flatten_layer(layer2)
  fc1 = fc_layer(layer_flat,
                n_feature,
                num_fc)
  fc1 = tf.nn.relu(fc1)
  fc2 = fc_layer(fc1,
                num_fc,
                num_class)  
  return fc2

In [0]:
def backward(mnist):
  x = tf.placeholder(tf.float32,shape=[None,img_size_flat])
  y_labels = tf.placeholder(tf.float32,[None,num_class])
  y_output = forward(x)
  #y = tf.nn.softmax(y_output)
  global_step = tf.Variable(0,trainable=False)
  LEARNING_RATE = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            mnist.train.num_examples/BATCH_SIZE,
            LEARNING_RATE_DECAY,
            staircase = True)
  
  a1 = tf.equal(tf.argmax(y_output,1),tf.argmax(y_labels, 1))
  acc1 = tf.reduce_mean(tf.cast(a1, tf.float32))
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_output,labels=y_labels)
  loss = tf.reduce_mean(cross_entropy) #+ tf.add_n(tf.get_collection("losses"))


  train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss,global_step=global_step)
  #train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(loss)
  #train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

  ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
  ema_op = ema.apply(tf.trainable_variables())
  with tf.control_dependencies([train_step,ema_op]):  
    train_op = tf.no_op(name="train")



  fd_test = {x:mnist.test.images,y_labels:mnist.test.labels}
  saver = tf.train.Saver()
  with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)  
    ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH) 
    if ckpt and ckpt.model_checkpoint_path:
      saver.restore(sess,ckpt.model_checkpoint_path)
    STEP =10000000
    for i in range(STEP):
      images,labels = mnist.test.next_batch(BATCH_SIZE)
      fd = {
          x: images,
          y_labels: labels
      }
      _,loss_ = sess.run([train_step,loss],feed_dict=fd)
      acc_t1 = sess.run(acc1,feed_dict=fd_test)
      if  i%10 == 0 :
        print('Train step:{0}K\tLoss:{1}\t Accuracy:{2:.2%}'
              .format(i/1000,loss_,acc_t1))
      if i%200 == 0 :
        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME))

    

In [0]:
backward(mnist)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model_save/cnn_mnist/cnn_mnist
Train step:0.0K	Loss:58759.6328125	 Accuracy:18.97%
Train step:0.01K	Loss:5100.216796875	 Accuracy:48.17%
Train step:0.02K	Loss:3107.819091796875	 Accuracy:61.52%
Train step:0.03K	Loss:1659.304931640625	 Accuracy:68.13%
Train step:0.04K	Loss:2295.419921875	 Accuracy:72.03%
Train step:0.05K	Loss:2277.982666015625	 Accuracy:73.80%
Train step:0.06K	Loss:1623.7926025390625	 Accuracy:77.14%
Train step:0.07K	Loss:1246.701171875	 Accuracy:78.88%
Train step:0.08K	Loss:1066.074462890625	 Accuracy:79.74%
Train step:0.09K	Loss:734.024658203125	 Accuracy:81.54%
Train step:0.1K	Loss:707.6424560546875	 Accuracy:82.52%
Train step:0.11K	Loss:553.4598388671875	 Accuracy:81.83%
Train step:0.12K	Loss:428.3624267578125	 Accuracy:83.76%
Train step:0.13K	

KeyboardInterrupt: ignored